# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119734e+02     1.553354e+00
 * time: 0.808974027633667
     1     1.148487e+01     9.427589e-01
 * time: 2.180215835571289
     2    -1.112810e+01     1.081656e+00
 * time: 2.8296148777008057
     3    -3.388743e+01     8.830804e-01
 * time: 3.7745649814605713
     4    -4.753827e+01     6.966062e-01
 * time: 4.751945972442627
     5    -5.674686e+01     2.333736e-01
 * time: 5.682877063751221
     6    -5.966489e+01     1.687723e-01
 * time: 6.319571018218994
     7    -6.081472e+01     5.345782e-02
 * time: 6.963544845581055
     8    -6.125106e+01     8.068274e-02
 * time: 7.621271848678589
     9    -6.155244e+01     5.540302e-02
 * time: 8.25409984588623
    10    -6.177077e+01     2.598607e-02
 * time: 8.88764500617981
    11    -6.194139e+01     1.925781e-02
 * time: 9.51869797706604
    12    -6.200737e+01     1.919691e-02
 * time: 10.162158966064453
    13    -6.207500e+01     1.507635e-02
 * time: 10.793640851974487
    14

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671068
    AtomicLocal         -18.8557718
    AtomicNonlocal      14.8522680
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485380 
    Xc                  -19.3336823

    total               -62.261666451454